<a href="https://colab.research.google.com/github/aikokass82-design/home-tasks-/blob/main/Lesson_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount("/content/drive")
from datetime import datetime

In [ ]:
advertising_info = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/reclamni_zosobi-info.csv")
advertising_info

In [ ]:
distributor_info = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/distributor-info.csv")
distributor_info

In [ ]:
#1 zadaniye
advertising_info.columns = advertising_info.columns.str.strip()
distributor_info.columns = distributor_info.columns.str.strip()

advertising_info.dropna(subset=["id", "distributorid"], inplace=True)
distributor_info.dropna(subset=["id"], inplace=True)

advertising_info.drop_duplicates(inplace=True)
distributor_info.drop_duplicates(inplace=True)

advertising_info['issuedDate'] = pd.to_datetime(advertising_info['issuedDate'], errors='coerce')
advertising_info['validFrom'] = pd.to_datetime(advertising_info['validFrom'], errors='coerce')
advertising_info['validThrough'] = pd.to_datetime(advertising_info['validThrough'], errors='coerce')
advertising_info['status'] = advertising_info['status'].str.lower().str.strip()
advertising_info['distributorid'] = advertising_info['distributorid'].astype(str)
distributor_info['id'] = distributor_info['id'].astype(str)

print(advertising_info.info())
print(distributor_info.info())

In [ ]:
today = pd.Timestamp.today()
advertising_info["isActiveNow"] = (advertising_info["validFrom"]<=today)&(advertising_info["validThrough"]>=today)
print(advertising_info[["id", "validFrom","validThrough", "isActiveNow"]].head(10))
print(advertising_info["isActiveNow"].value_counts())

In [ ]:
!pip install ydata-profiling

In [ ]:
import ydata_profiling
profile_adv = ydata_profiling.ProfileReport(advertising_info)
profile_adv.to_file("advertising_info_report.html")

profile_dist = ydata_profiling.ProfileReport(distributor_info)
profile_dist.to_file("distributor_info_report.html")

2 zadaniye

In [ ]:
#1
type_counts = advertising_info["type"].value_counts()
print(type_counts.head(5))

In [ ]:
#2
pivo_type_district = advertising_info.pivot_table(index="district", columns="type", values="id", aggfunc="count", fill_value=0)
print(pivo_type_district)

In [ ]:
#3
advertising_info = advertising_info.merge(distributor_info, left_on='distributorid', right_on='id', how='left')
advertising_info.rename(columns={'distributor': 'company'}, inplace=True)
company_counts = advertising_info['company'].value_counts()
print(company_counts)

In [ ]:
#4
company_area = advertising_info.groupby("company")["area"].sum().sort_values(ascending=False)
print(company_area.head(10))

In [ ]:
#5
pivot_area_district = advertising_info.pivot_table(index="district", columns="company", values="area", aggfunc="sum", fill_value=0)
print(pivot_area_district)

In [ ]:
#6
today=pd.Timestamp.today()
closest_expiry = advertising_info.sort_values("validThrough").head(10)
print(closest_expiry[["company", "validThrough"]])

In [ ]:
#7
company_counts = advertising_info["company"].value_counts()
top5_companies = company_counts.head(5).index
top5_data=advertising_info[advertising_info["company"].isin(top5_companies)]
pivot_top5_type = top5_data.pivot_table(index="company", columns="type", aggfunc="size", fill_value=0)
print(pivot_top5_type)

In [ ]:
top_type_per_company = pivot_top5_type.idxmax(axis=1)
print("Наиболее предпочтительный тип рекламы у топ5 компаний:")
print(top_type_per_company)

Zadaniye 3

In [ ]:
#1
adv_counts = advertising_info["district"].value_counts()
adv_area = advertising_info.groupby("district")["area"].sum().sort_values(ascending=False)
print("TOP 5 районов по кол-ву реклам:")
print(adv_counts.head(5))
print("\nTOP 5 райнов по суммарной площади рекламы:")
print(adv_area.head(5))

Оболонський район является наиболее перспективным, так как здесь сосредоточено большое количество рекламных объектов и компании. Высокая доля рекламных площадей говорит о высоком спросе и конкурентной активности.

In [ ]:
#2
chosen_area = "Оболонський"
leaders = (
   advertising_info[advertising_info["district"]==chosen_area]["company"].value_counts().head(3)
)
print("Топ 3 компании в Оболонськом районе:")
print(leaders)

In [ ]:
#3
top3_companies = leaders.index
top3_num=advertising_info[advertising_info["company"].isin(top3_companies)&(advertising_info["district"]==chosen_area)]
pivot_top3= top3_num.pivot_table(
    index="company", columns="type", aggfunc="size", fill_value=0
)
pivot_top3.plot(kind = "bar", stacked=True, title=f"Типы рекламы у лидеров в районе {chosen_area}", figsize=(8,5))

In [ ]:
#4
import seaborn as sns
import matplotlib.pyplot as plt
advertising_info["validFrom"]=pd.to_datetime(advertising_info["validFrom"], errors = "coerce")
advertising_info["year"] = advertising_info["validFrom"].dt.year
sns.lineplot(data=advertising_info, x="year", y="area", estimator="mean", marker="o")
plt.title("Средняя площадь рекламы по годам")
plt.ylabel("Средняя площадь (m^2)")
plt.show()

In [ ]:
#5
chosen_company = top3_companies[0]
company_data = advertising_info[advertising_info["company"]==chosen_company]
company_year = company_data.groupby("year").size()
company_year.plot(kind="bar", title = f"Кол-во рекламы по годам ({chosen_company})", figsize=(8,4))

company_types = company_data["type"].value_counts()
plt.figure(figsize=(5,5))
plt.pie(company_types, labels=company_types.index, autopct="%1.1f%%")
plt.title(f"Распределение типов рекламы компании {chosen_company}")
plt.show()

Zadaniye 4

На основании анализа количества рекламных объектов и суммарной площади наиболее активным районом оказался Оболонський. Это значит, что там высокий спрос на наружную рекламу и много компаний конкурируют за внимание аудитории. Рекомендация: сосредоточиться на этом районе, так как там больше потенциальных клиентов.

По данным топ3 компаний в этом районе, наиболее популярные типы рекламы: «Спеціальна рекламна конструкція на фасаді будинку (будівлі)» и «Щит на фасаді будинку (будівлі)». Это значит, что компании активно используют именно эти форматы. При возможности можно предложить и другие современные решения, например LED/цифровые экраны.

Компании, у которых срок действия договора заканчивается ближе всего к текущему моменту, скоро будут продлевать или искать новых поставщиков. По данным анализа (пункт 6 задания 2) такие компании можно выделить и предложить им услуги по обновлению рекламы или расширению форматов.